## СОЗДАНИЕ ЦИФРОВОГО НЕФТЕГАЗОВОГО ЭКСПЕРТА

Задача участников заключается в разработке модели-консультанта для нефтегазовой отрасли, используя существующие большие языковые модели (БЯМ) с открытым кодом и предобученными весами.

Тестовые и валидационные данные представляют собой вопросы на русском языке из различных областей нефтегазовой сферы: геологии, геофизики, минералогии и бурения.

Участникам предоставляется валидационный набор данных (в виде массива), содержащий 15 вопросов и ответов на них из заданной тематики.

Тестирование разработанных алгоритмов будет проводиться на скрытых тестовых вопросах, которые схожи по тематике с предоставленными для валидации.

Качество ответов моделей на вопросы с выбором ответа будет оцениваться по метрике Recall (доля правильных ответов).

Участником разрешено использовать только локальные модели с общим числом параметров, не превосходящим 10 млрд. Разрешено выполнять дообучение и использовать промт-инжиниринг. Разрешено использовать любые данные для дообучения, в том числе из сети Интернет. Однако, при тестировании использование сети Интернет строго запрещено.

**Время работы программы в Google Colab на тестовых вопросах не должно превышать 2 часа.**

Разрешено использовать только одну модель для обоих типов вопросов: с выбором ответа и с развёрнутым ответом.

Ниже представлено базовое решение для знакомтсва с задачей

По всем вопросам: @goshutka

In [1]:
#########################################################
#
# ПОЖАЛУЙСТА НЕ МЕНЯЙТЕ СТРУКТУРУ МАССИВОВ И НАЗВАНИЯ
# ОСНОВНЫХ ПЕРЕМЕННЫХ
# ВСЕ ДАЛЬНЕЙШИЕ МАНИПУЛЯЦИИ ВЫПОЛНЯЙТЕ С ПЕРЕМЕННЫМИ:
#
# val_questions, val_questions_ans_variants, val_questions_true_ans
# LLM_answer_arr, val_qustions_detailed, val_qustions_detailed_true_ans, LLM_answer_arr_detailed
#
# ИТОГОВОЕ ТЕСТИРОВАНИЕ БУДЕТ ПРОВОДИТЬСЯ С ЭТИМИ ПЕРЕМЕННЫМИ,
# НО С ДРУГИМИ ВОПРОСАМИ
#
#########################################################


#Валидационный набор данных
val_questions = [
    'Поверхность Мохоровичича - это:',
    'Процесс вытаивания подземных льдов носит название:',
    'Как называется слой верхней мантии, в котором находятся очаги современных вулканов и протекают процессы дифференциации ее вещества:',
    'Лёсс - это:',
    'Кто является автором гелиоцентрической теории происхождения является:',

    'Какие из этих минералов царапают стекло:',
    'К числу главнейших свойств минералов не относится:',
    'Минералы группы слюд обладают:',
    'Сколько направлений спайности у флюорита:',
    'По какому свойству отличаются пирит и халькопирит:',

    'Гравиразведка широко применяется для поисков:',
    'Какой из геофизических методов наиболее результативен при поисках крупных месторождений железных руд:',
    'Наибольшей плотностью из ниже перечисленного обладает:',
    'Гравиразведка направлена на:',
    'К основным компонентам магнитного поля не относится:',
]

#Варианты ответов
val_questions_ans_variants = [

    "1) Граница мантия-ядро, 2) Граница океан - суша, 3) Граница земная кора - мантия, 4) Граница внутреннего ядра и внешнего",
    "1) Курумы, 2) Термокарст, 3) Ликвация, 4) Солифлюкция",
    "1) Поверхность Конрада, 2) Астеносфера, 3) Литосфера, 4) Андезитовый слой",
    "1) Рыхлая пористая горная порода серовато-желтого, или буровато-желтого цвета, в составе которой преобладают - более 50% - алевритовые частицы, 2) Неправильное написание слова Лес, 3) Горная порода, образованная в результате разрушения магматических горных пород в лесных массивах Амазонки, 4) Рыхлая горная порода, сложенная исключительно зернами кварца пылеватой размерности (менее 0,01 мм)",
    "1) Николай Коперник, 2) Ж.Л.Леклерк де Бюффон, 3) Иммануил Кант, 4) Пьер Симон Лаплас",

    "1) Тальк, гипс, кальцит, 2) Флюорит, галенит, молибденит, 3) Гранат, турмалин, касситерит, 4) Сфалерит, антимонит, вивианит",
    "1) Блеск, 2) Твердость, 3) Спайность, 4) Пьезоэлектрические свойства",
    "1) Весьма совершенной спайностью, 2) Высокой твёрдостью, 3) Игольчатой формой кристаллов, 4) Хорошей растворимостью",
    "1) 2, 2) 4, 3) 3, 4) 6",
    "1) По твердости, 2) По спайности, 3) По блеску, 4) По магнитности",

    "1) Железорудных и хромитовых месторождений, 2) Кварцевых жил, 3) Гравия, 4) Гравиразведка применяется только для картирования разрывных нарушений и изучения поля силы тяжести.",
    "1) Электроразведка, 2) Магниторазведка, 3) Разведка боем, 4) Феррометрия",
    "1) Вода, 2) Гранит, 3) Песок, 4) Глина",
    "1) Открытие месторождения строительных материалов, 2) Выявлений аномалий поля силы тяжести, 3) Графологический анализ узора речной сети, 4) Уточнение величины постоянной всемирного тяготения",
    "1) Магнитное склонение, 2) Магнитное наклонение, 3) Напряженность магнитного поля, 4) Отклонение магнитного поля"

]

# Верные номера ответов
val_questions_true_ans = [
    3,
    2,
    2,
    4,
    1,

    3,
    4,
    1,
    2,
    1,

    1,
    2,
    2,
    2,
    4,
]

# Верные ответы

#    'Граница земная кора - мантия',
#    'Термокарст',
#    'Астеносфера',
#    'Рыхлая горная порода, сложенная исключительно зернами кварца пылеватой размерности (менее 0,01 мм)',
#    'Николай Коперник',

#    'Гранат, турмалин, касситерит',
#    'Пьезоэлектрические свойства',
#    'Весьма совершенной спайностью',
#    '4',
#    'По твердости',

#    'Железорудных и хромитовых месторождений',
#    'Магниторазведка',
#    'Гранит',
#    'Выявлений аномалий поля силы тяжести',
#    'Отклонение магнитного поля',

val_qustions_detailed = [

    "Какой минерал с химическим составом CaMg(CO3)2?",
    "Какой минерал используется для производства керамики и фарфора?",
    "Как называется противовыбросовое оборудование, применяемое при бурении?",
    "Скопления какого газа широко представлены на дне Черного моря?",
    "Для какого периода характерно пышное развитие древесной наземной растительности, распространение насекомых, появление первых рептилий?"
]

val_qustions_detailed_true_ans = [

    "Доломит",
    "Каолинит",
    "Превентор",
    "Сероводорода",
    "Каменноугольного"

]

LLM_answer_arr = []

LLM_answer_arr_detailed = []

In [ ]:
# Загрузка модели с Hugging Face
import numpy as np
from huggingface_hub import login
access_token  = "___"
login(access_token)

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import time
import pandas as pd
import json
from sentence_transformers import SentenceTransformer, util

model_id = "google/gemma-2-2b-it"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cpu",
    torch_dtype=dtype,)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Вопросы с выбором ответа**

In [4]:
# Формирование запросов к БЯМ

retriever = SentenceTransformer("intfloat/multilingual-e5-base")
passages = [json.loads(line)["text"] for line in open("/content/drive/MyDrive/Task1/geology_terms.jsonl", encoding="utf-8")]
passage_embeddings = retriever.encode(passages, convert_to_tensor=True)

start_time = time.time()

for i in range(len(val_questions)):
  full_query = f"{val_questions[i]}\nВарианты: {val_questions_ans_variants[i]}"
  query_embedding = retriever.encode("query: " + full_query, convert_to_tensor=True)
  hits = util.semantic_search(query_embedding, passage_embeddings, top_k=5)[0]
  retrieved_texts = [passages[hit["corpus_id"]] for hit in hits]
  context = "\n".join(retrieved_texts)

  chat = [
    { "role": "user", "content": f"Контекст:\n{context}\n\nДай ответ на вопрос: {val_questions[i]}. Выбери один ответ из нескольких вариантов: {val_questions_ans_variants[i]}. В ответе укажи только одну цифру, вариант ответа." },
  ]

  prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
  inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
  outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=30)

  decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(f"\nВопрос {i+1} — сырой ответ модели:\n{decoded}")
  print("_" * 50)

  LLM_ans = tokenizer.decode(outputs[0]).split('model')[1].split('\n')[1]

  #Проверка, что БЯМ возвращает число
  try:
    LLM_ans = int(LLM_ans)
  except:
    LLM_ans = -9999

  LLM_answer_arr.append(LLM_ans)

RECALL = np.sum(np.array(LLM_answer_arr) == np.array(val_questions_true_ans)) / len(val_questions_true_ans)
print(f'Итоговая метрика вопросов с контекстом (RAG): {RECALL}')

end_time = time.time()
print(f"Время обучения c выбором ответа: {end_time - start_time:.2f} сек.")


Вопрос 1 — сырой ответ модели:
user
Контекст:
Поверхность Мохоровичича - граница между земной корой и литосферной мантией, на которой происходит резкое увеличение скоростей продольных и поперечных сейсмических волн.
Граница Мохоровичича - граница между земной корой и литосферной мантией, на которой происходит резкое увеличение скоростей продольных и поперечных сейсмических волн.
Земная кора - внешняя твёрдая оболочка Земли, имеющая мощность до 10 км (океанская) или до 45 км (континентальная).
Континентальная кора - земная кора, залегающая под континентами и многими крупными островами, имеющая мощность до 40-45 км, состоящая из осадочного, гранито-гнейсового и гранулито-базитового слоёв.
Конвергентная граница - граница взаимного схождения по горизонтали относительно друг друга двух соседних литосферных плит.

Дай ответ на вопрос: Поверхность Мохоровичича - это:. Выбери один ответ из нескольких вариантов: 1) Граница мантия-ядро, 2) Граница океан - суша, 3) Граница земная кора - мантия, 

In [5]:
df_qa = pd.DataFrame({
    "Вопрос": val_questions,
    "Варианты ответов": val_questions_ans_variants,
    "Правильный ответ": val_questions_true_ans,
    "Ответ модели": LLM_answer_arr,
    "Совпадает?": [model == true for model, true in zip(LLM_answer_arr, val_questions_true_ans)]
})

pd.set_option('display.max_colwidth', 200)
df_qa.style.applymap(lambda x: 'background-color: lightgreen' if x is True else 'background-color: pink', subset=['Совпадает?'])

<ipython-input-5-3ce553380750>:10: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_qa.style.applymap(lambda x: 'background-color: lightgreen' if x is True else 'background-color: pink', subset=['Совпадает?'])


,Вопрос,Варианты ответов,Правильный ответ,Ответ модели,Совпадает?
0,Поверхность Мохоровичича - это:,"1) Граница мантия-ядро, 2) Граница океан - суша, 3) Граница земная кора - мантия, 4) Граница внутреннего ядра и внешнего",3,3,True
1,Процесс вытаивания подземных льдов носит название:,"1) Курумы, 2) Термокарст, 3) Ликвация, 4) Солифлюкция",2,2,True
2,"Как называется слой верхней мантии, в котором находятся очаги современных вулканов и протекают процессы дифференциации ее вещества:","1) Поверхность Конрада, 2) Астеносфера, 3) Литосфера, 4) Андезитовый слой",2,2,True
3,Лёсс - это:,"1) Рыхлая пористая горная порода серовато-желтого, или буровато-желтого цвета, в составе которой преобладают - более 50% - алевритовые частицы, 2) Неправильное написание слова Лес, 3) Горная порода, образованная в результате разрушения магматических горных пород в лесных массивах Амазонки, 4) Рыхлая горная порода, сложенная исключительно зернами кварца пылеватой размерности (менее 0,01 мм)",4,4,True
4,Кто является автором гелиоцентрической теории происхождения является:,"1) Николай Коперник, 2) Ж.Л.Леклерк де Бюффон, 3) Иммануил Кант, 4) Пьер Симон Лаплас",1,1,True
5,Какие из этих минералов царапают стекло:,"1) Тальк, гипс, кальцит, 2) Флюорит, галенит, молибденит, 3) Гранат, турмалин, касситерит, 4) Сфалерит, антимонит, вивианит",3,1,False
6,К числу главнейших свойств минералов не относится:,"1) Блеск, 2) Твердость, 3) Спайность, 4) Пьезоэлектрические свойства",4,4,True
7,Минералы группы слюд обладают:,"1) Весьма совершенной спайностью, 2) Высокой твёрдостью, 3) Игольчатой формой кристаллов, 4) Хорошей растворимостью",1,1,True
8,Сколько направлений спайности у флюорита:,"1) 2, 2) 4, 3) 3, 4) 6",2,2,True
9,По какому свойству отличаются пирит и халькопирит:,"1) По твердости, 2) По спайности, 3) По блеску, 4) По магнитности",1,1,True


**Вопросы с развёрнутым ответом**

In [6]:
# Формирование запросов к БЯМ
start_time = time.time()

for i in range(len(val_qustions_detailed_true_ans)):
  full_query = f"{val_qustions_detailed[i]}"
  query_embedding = retriever.encode("query: " + full_query, convert_to_tensor=True)
  hits = util.semantic_search(query_embedding, passage_embeddings, top_k=5)[0]
  retrieved_texts = [passages[hit["corpus_id"]] for hit in hits]
  context = "\n".join(retrieved_texts)

  chat = [
  { "role": "user", "content": f"Контекст:\n{context}\n\nДай ответ на вопрос: {val_qustions_detailed[i]}. Хорошо подумай и ответь одной фразой. " },
  ]

  prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
  inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
  outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=100)

  decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(f"\nВопрос {i+1} — сырой ответ модели:\n{decoded}")
  print(f"\nПравильный ответ - {val_qustions_detailed_true_ans[i]}")
  print("_" * 50)

  LLM_ans = tokenizer.decode(outputs[0]).split('model')[1].split('\n')[1]

  LLM_answer_arr_detailed.append(LLM_ans)

end_time = time.time()
print(f"Время обучения c развернутым ответом: {end_time - start_time:.2f} сек.")


Вопрос 1 — сырой ответ модели:
user
Контекст:
Кальцит — карбонат кальция (CaCO₃), один из самых распространённых минералов земной коры. Входит в состав известняка, мрамора и других осадочных и метаморфических пород.
Доломит - 1) породообразующий минерал класса карбонатов, CaMg(CO~3~)~2~ белого, сероватого цвета; 2) осадочная карбонатная горная порода, целиком или преимущественно состоящая из минерала доломита.
Карбонатная порода - горная порода, состоящая из минералов кальцита, доломита, магнезита, сидерита и примесей.
Мрамор - метаморфическая горная порода, состоящая только из кальцита CaCO~3~.
Гранит - магматическая интрузивная кислая порода, содержащая 65-78% SiO~2~, состоящая из кварца, плагиоклаза и полевого шпата.

Дай ответ на вопрос: Какой минерал с химическим составом CaMg(CO3)2?. Хорошо подумай и ответь одной фразой.
model
Минералом с химическим составом CaMg(CO3)2 является доломит. 


Правильный ответ - Доломит
__________________________________________________

Вопрос 2 — 

**Оценка качества (необязательная часть)**

In [ ]:
#################################################################
#
#ПОЖАЛУЙСТА, НЕ МЕНЯЙТЕ И НЕ УДАЛЯЙТЕ ЭТУ ЯЧЕЙКУ
#ЗДЕСЬ ПРОИЗВОДИТСЯ ФИНАЛЬНЫЙ РАСЧЁТ МЕТРИКИ КАЧЕСТВА ОТВЕТОВ МОДЕЛИ
#ВЫ МОЖЕТЕ ВЫПОЛНИТЬ ЕГО САМОСТОЯТЕЛЬНО, ЕСЛИ ОФОРМИТЕ ДОСТУП К API
#
#               ИНСТРУКЦИЯ ДЛЯ ПОЛУЧЕНИЯ API КЛЮЧА (по желанию)
# 1) ЗАРЕГИСТРИРОВАТЬСЯ НА САЙТЕ https://gen-api.ru/model/deepseek-v3
# 2) ВЫБРАТЬ ВКЛАДКУ API
# 3) ВЫБРАТЬ ПОЛУЧЕНИЕ API КЛЮЧА
#
################################################################

import requests
# ЗДЕСЬ НУЖНО УКАЗАТЬ СВОЙ УНИКАЛЬНЫЙ API КЛЮЧ (по желанию)
api_key = '___'
#
url_endpoint = "https://api.gen-api.ru/api/v1/networks/deepseek-v3"

input = {
    "is_sync": True,
    "messages": [
        {
            "role": "user",
            "content": ""
        }
    ],
    "model": "deepseek-v3",
    "stream": False,
    "max_tokens": 4096,
    "temperature": 1,
    "top_p": 1
}
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': f'Bearer {api_key}'
 }

scores = []
for k in range(len(val_qustions_detailed_true_ans)):
  input['messages'][0]['content'] = f'Оцени корректность (0-40 баллов), полноту (0-30 баллов) и точность (0-30 баллов) и выдай итоговую оценку ответа: {LLM_answer_arr_detailed[k]} от 0 до 100 баллов на вопрос: {val_qustions_detailed[k]} с учетом того, что правильный ответ — {val_qustions_detailed_true_ans[k]}. Выдавай только итоговую оценку в виде целого числа от 0 до 100 без лишних слов.'
  response = requests.post(url_endpoint, json=input, headers=headers)
  score = float(response.json()['response'][0]['choices'][0]['message']['content'])
  scores.append(score/100)

print(f'Итоговая метрика вопросов с выбором ответа составила:{RECALL}')
print(f'Итоговая метрика вопросов с развёрнутым ответом составила:{np.mean(scores)}')
print(f'Итоговая метрика:{2/(1/RECALL+1/np.mean(scores))}')

Итоговая метрика вопросов с выбором ответа составила:0.9333333333333333
Итоговая метрика вопросов с развёрнутым ответом составила:1.0
Итоговая метрика:0.9655172413793105
